In [2]:
#######################################################
# Created-Date: 04/15/2016                            #
# Last Modified: 04/27/2016                           #
# Created by: Avineesh Vasudevan , Shravan Kumar ,    #
#             Asish Mahapatra                         #
#                                                     #
# Objective: Build models to predict the song hit rate#
#            based on KNN and Trees classifier models #
#                                                     #
#                                                     #
# Instruction before running:                         #
# 1. Please change the file path of directory         # 
# 2. change ranges based on the number of lyrics      # 
#    you have in the directory                        #  
# 3. Please follow similar nomenclature as used in the# 
#    code                                             #
# 4. Please use the array list of only the years      #
#    under consideration and comment the rest         #
# 5. Please use the scenario definition to assign 0's #
#    to the relevant subset of the super set          #
#######################################################

# Importing required libraries
import pandas as pd
import matplotlib.pyplot as plt
import pandas.io.data
from statsmodels import regression
import math
import statsmodels.api as sm
import datetime
import numpy 
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve, auc


C:\Users\avinesh\Anaconda3\lib\site-packages\pandas\io\data.py:33: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [3]:
# Building and Analyzing Data Scenario 1
dhome = "/Users/avinesh/Google Drive/Courseworks/2 Sem/Python-based Data Analytics/Python Project/All Data/fsongs"
year = ['2030','2020']
# Adding label 1 to 2020 Dataset
filepath = dhome+year[0]+".csv"
df=pd.read_csv(filepath)
grades = []


In [4]:
for row in df['ranks']:
    grades.append(1)
    
df['grades'] = grades


In [5]:
len(df['grades'])

100

In [6]:

# Adding label 0 to 2030 Dataset
filepath = dhome+year[1]+".csv"
df1=pd.read_csv(filepath)
grades = []
for row in df1['ranks']:
    grades.append(0)

df1['grades'] = grades

df = df.append(df1)




In [8]:
# Dropping Duplicates
od = df.sort(['songs','artists'], ascending=[1,0])
ot=od.drop_duplicates(['songs','artists'], keep='first')
print(ot.shape)
ot.pop('ranks')
ot.pop('ids')
ot.pop('artists')
ot.pop('songs')
ot['grades'].value_counts()

(202, 23)


C:\Users\avinesh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


0    102
1    100
Name: grades, dtype: int64

In [10]:
# Splitting data into train and test data
from sklearn.cross_validation import train_test_split
df_train,df_test = train_test_split(ot,random_state=0)

print(df_train.shape)
print(df_test.shape)


(151, 19)
(51, 19)
1    78
0    73
Name: grades, dtype: int64
0    29
1    22
Name: grades, dtype: int64


In [11]:
# Logistic Regression Without PCA
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
y = df_train['grades']
df_train.pop('grades')
model_lr = LogisticRegression()
model_lr.fit(df_train, y)  
print(model_lr.score(df_train, y))

0.675496688742


In [12]:
# Predicting Logistic Regression on Testing Dataet
z = df_test['grades']
df_test.pop('grades')
fit = model_lr.predict(df_test)

In [13]:
# Summary Statistics for Logistic Regression
print(metrics.classification_report(fit, z))
print(metrics.accuracy_score(fit, z))     
print(metrics.confusion_matrix(fit, z))

             precision    recall  f1-score   support

          0       0.45      0.54      0.49        24
          1       0.50      0.41      0.45        27

avg / total       0.48      0.47      0.47        51

0.470588235294
[[13 11]
 [16 11]]


In [19]:
model_lr.coef_

array([[  2.09063048e-01,  -3.52035691e-01,   9.10697272e-04,
          1.52334429e+00,   2.28745140e-02,  -2.32349170e-02,
          7.19249664e-02,   8.76410185e-01,  -3.33220700e-01,
         -6.34637320e-03,  -4.19162508e-01,   4.72013593e-01,
         -4.19126147e-03,  -3.70828056e-02,  -1.65969093e-01,
          2.99823665e-01,  -3.29563406e-01,  -1.27682501e-01]])

In [47]:


probs = model_lr.predict_proba(df_test)
print(probs)
print(metrics.roc_auc_score(fit, probs[:, 1]))
print(len(probs))

Area under the ROC curve : 0.500000


In [48]:
# Evaluating the model using cross-validation(using 10-Fold)
md_Y = ot['grades']
ot.pop('grades')
md_X = ot

In [49]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(LogisticRegression(), md_X, md_Y, scoring='accuracy', cv=10)
print (scores)
print (scores.mean())

[ 0.79347826  0.79347826  0.79347826  0.79347826  0.79120879  0.8         0.8
  0.8         0.8         0.8       ]
0.796512183469


In [ ]:
df_train,df_test = train_test_split(ot,random_state=0)

print(df_train.shape)
print(df_test.shape)
print(df_train['grades'].value_counts())
print(df_test['grades'].value_counts())

In [ ]:
#implementing trees

In [33]:
# Splitting into independent and dependent variables
ys1, xs1 = dmatrices('grades ~ acousticness + danceability + duration + energy + key + liveness + loudness + mode + speechiness + tempo + time_signature + valence + Anger + Surprise + Love + Joy + Sadness + Fear', 
                   data = ot, return_type="dataframe")
ys1 = numpy.ravel(ys1)


In [37]:
xs1.pop('Intercept')

19    1
13    1
65    1
17    1
87    1
70    1
40    1
62    1
71    1
97    1
8     1
2     1
15    1
80    1
19    1
14    1
84    1
48    1
54    1
55    1
7     1
6     1
35    1
37    1
98    1
55    1
47    1
49    1
64    1
29    1
     ..
77    1
1     1
20    1
46    1
12    1
78    1
11    1
40    1
91    1
96    1
10    1
25    1
26    1
62    1
66    1
82    1
72    1
16    1
74    1
39    1
13    1
8     1
18    1
88    1
83    1
92    1
68    1
6     1
16    1
72    1
Name: Intercept, dtype: float64

In [38]:
xs1

,acousticness,danceability,duration,energy,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,Anger,Surprise,Love,Joy,Sadness,Fear
19,0.074356,0.517933,393.60000,0.492540,1,0.069184,-7.396,1,0.027861,131.189,4,0.235759,1.261152,1.308458,1.436487,1.319803,1.268168,1.319803
13,0.074356,0.517933,393.60000,0.492540,1,0.069184,-7.396,1,0.027861,131.189,4,0.235759,1.275426,1.323098,1.451504,1.334373,1.283369,1.334373
65,0.473817,0.511773,242.36000,0.823738,9,0.080829,-7.269,1,0.273506,148.063,3,0.183434,1.645729,1.748974,2.039133,1.784766,1.519658,1.784766
17,0.004035,0.682043,201.77333,0.987871,2,0.283666,-3.859,1,0.035419,140.037,4,0.794043,1.512821,1.669518,1.769517,1.628808,1.446962,1.613404
87,0.007207,0.784116,306.29288,0.664036,9,0.158843,-8.482,0,0.034874,127.549,4,0.961100,1.556438,1.616655,1.779251,1.620672,1.545891,1.620672
70,0.068485,0.806061,188.02667,0.855451,9,0.116679,-5.313,1,0.054052,134.071,4,0.963923,0.943403,0.965049,1.025678,0.967949,0.950549,0.967949
40,0.096857,0.927517,215.33333,0.530833,5,0.160141,-6.520,0,0.161296,109.963,4,0.764963,0.863097,0.922463,1.020759,0.922463,0.858938,0.922463
62,0.038555,0.657365,184.61333,0.558068,8,0.091263,-7.474,1,0.029434,118.017,4,0.123847,0.945770,1.008360,1.110338,1.008360,0.907751,1.008360
71,0.165766,0.164260,287.37333,0.724350,7,0.046118,-5.699,1,0.039023,90.960,4,0.417742,1.054075,1.106742,1.209449,1.110920,1.067122,1.110920
97,0.970227,0.309855,25.05098,0.037436,9,0.111845,-17.672,0,0.033626,84.045,3,0.522703,1.437985,1.547337,1.717263,1.547707,1.375704,1.547707


In [46]:
# Splitting into test and training data
ys1_train, ys1_test, xs1_train, xs1_test = train_test_split(ys1, xs1, random_state=0)

In [47]:
model_s1tr = tree.DecisionTreeClassifier()
model_s1tr.fit(xs1_train, ys1_train) 
predicts_s1tr = model_s1tr.predict(xs1_test)

In [48]:
print(metrics.classification_report(predicts_s1tr, ys1_test))
print(metrics.accuracy_score(predicts_s1tr, ys1_test))     
print(metrics.confusion_matrix(predicts_s1tr, ys1_test))

             precision    recall  f1-score   support

        0.0       0.52      0.52      0.52        29
        1.0       0.36      0.36      0.36        22

avg / total       0.45      0.45      0.45        51

0.450980392157
[[15 14]
 [14  8]]


In [52]:
s1tr_fpr, s1_trtpr, thresholds = roc_curve(ys1_test, predicts_s1tr ,pos_label=1) 
s1tr_roc_auc = auc(s1tr_fpr, s1_trtpr)
s1tr_roc_auc


0.4404388714733542

In [ ]:
#Implemeting KNN

In [68]:
model_s1nn = KNeighborsClassifier(n_neighbors = 50)
model_s1nn.fit(xs1_train, ys1_train) 
predicts_s1nn = model_s1nn.predict(xs1_test )
metrics.accuracy_score(ys1_test, predicts_s1nn)

0.60784313725490191

In [70]:
metrics.confusion_matrix(ys1_test, predicts_s1nn)

array([[20,  9],
       [11, 11]])

In [71]:
s1nn_fpr, s1_nntpr, thresholds = roc_curve(ys1_test, predicts_s1nn ,pos_label=1) 
s1nn_roc_auc = auc(s1nn_fpr, s1_nntpr)
s1nn_roc_auc


0.59482758620689657

In [ ]:
# Building and analyzing Data Scenario 2

In [72]:
dhome = "/Users/avinesh/Google Drive/Courseworks/2 Sem/Python-based Data Analytics/Python Project/All Data/fsongs"
year = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2020']

# Adding label 1 to 2020 Dataset
filepath = dhome+year[10]+".csv"
s2df=pandas.read_csv(filepath)
grades = []
for row in s2df['ranks']:
    if  row <= 103:
        grades.append(1)
    else:
        grades.append(0)
s2df['grades'] = grades


In [73]:
s2df

,ranks,songs,artists,ids,acousticness,danceability,duration,energy,key,liveness,...,tempo,time_signature,valence,Anger,Surprise,Joy,Sadness,Love,Fear,grades
0,1,THE TWIST,Chubby Checker,SOKPTVF13725C7344B,0.035865,0.466492,159.55592,0.647644,4,0.056933,...,155.230,4,0.966031,0.727710,0.783294,0.783294,0.715073,0.903446,0.783294,1
1,2,SMOOTH,santana,SOLHYBC135CC0B27FE,0.170286,0.620632,298.06667,0.922940,9,0.093313,...,115.952,4,0.962458,1.357096,1.428605,1.437286,1.335036,1.570031,1.437286,1
2,3,MACK THE KNIFE,Bobby Darin,SOYAUIR1376B0571F2,0.701482,0.546443,178.62531,0.852067,8,0.905568,...,95.200,4,0.634315,1.605895,1.684853,1.684853,1.632401,1.826029,1.684853,1
3,4,HOW DO I LIVE,LeAnn Rimes,SODLOLZ144912CCD9E,0.005334,0.764592,237.26621,0.908486,11,0.058760,...,127.972,4,0.886589,0.910407,0.981885,0.984372,0.879784,1.101369,0.984372,1
4,5,PARTY ROCK ANTHEM,lmfao,SOOIYXQ14373B73954,0.060660,0.683526,376.63156,0.501430,5,0.105768,...,130.033,4,0.182814,1.259442,1.340614,1.340614,1.182032,1.519476,1.340614,1
5,6,I GOTTA FEELING,The Black Eyed Peas,SOBTUCX146012A297D,0.083967,0.743047,289.13197,0.740592,0,0.421652,...,127.954,4,0.652845,1.439891,1.537006,1.537006,1.410294,1.695163,1.537006,1
6,7,macarena,Los Del Rio,SOVWGXW13D531FB427,0.005155,0.609297,220.31628,0.926622,0,0.163401,...,134.995,4,0.773814,1.182268,1.159297,1.261077,1.173461,1.283312,1.210187,1
7,8,PHYSICAL,Olivia Newton-John,SODITYF135C40313F7,0.100817,0.827085,222.62667,0.830078,4,0.109286,...,124.285,4,0.806108,1.004527,1.095194,1.095194,0.931961,1.239927,1.095194,1
8,9,YOU LIGHT UP MY LIFE,Debby Boone,SORVCNN13737B9E003,0.855203,0.322341,218.93333,0.282051,2,0.162640,...,77.470,3,0.078563,1.238923,1.281668,1.287147,1.261322,1.376655,1.287147,1
9,10,HEY JUDE,The Beatles,SOKMJYC13678934206,0.028010,0.402092,431.33333,0.551766,10,0.092866,...,145.164,4,0.651103,0.805659,0.850873,0.850873,0.798673,0.922265,0.850873,1


In [74]:
# Adding label 0 to Remaining Dataset
for i in range(0,10):
    filepath = dhome+year[i]+".csv"
    mds = pandas.read_csv(filepath,header=0)
    grades = []
    for row in mds['ranks']:
        if  row <= 101:
            grades.append(0)
        else:
            grades.append(1)
    mds['grades'] = grades
    s2df = s2df.append(mds)

In [82]:
s2df['grades']
len(s2df['grades'][99])

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
     ..
70    0
71    0
72    0
73    0
74    0
75    0
76    0
77    0
78    0
79    0
80    0
81    0
82    0
83    0
84    0
85    0
86    0
87    0
88    0
89    0
90    0
91    0
92    0
93    0
94    0
95    0
96    0
97    0
98    0
99    0
Name: grades, dtype: int64

In [83]:
# Dropping Duplicates
s2od = s2df.sort(['songs','artists'], ascending=[1,0])
s2ot = s2od.drop_duplicates(['songs','artists'], keep='first')
s2ot.shape
s2ot.columns.values


C:\Users\avinesh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


array(['ranks', 'songs', 'artists', 'ids', 'acousticness', 'danceability',
       'duration', 'energy', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'Anger',
       'Surprise', 'Joy', 'Sadness', 'Love', 'Fear', 'grades'], dtype=object)

In [84]:
# Splitting into dependent and independent varialbes 
ys2, xs2 = dmatrices('grades ~ acousticness + danceability + duration + energy + key + liveness + loudness + mode + speechiness + tempo + time_signature + valence + Anger + Surprise + Love + Joy + Sadness + Fear', 
                   data = s2ot, return_type="dataframe")
ys2 = numpy.ravel(ys2)

In [86]:
xs2.pop('Intercept')

19    1
65    1
96    1
60    1
64    1
77    1
77    1
70    1
89    1
86    1
22    1
85    1
80    1
40    1
20    1
91    1
55    1
42    1
50    1
86    1
87    1
54    1
96    1
99    1
96    1
46    1
31    1
5     1
32    1
70    1
     ..
6     1
31    1
80    1
38    1
81    1
16    1
79    1
40    1
13    1
73    1
69    1
6     1
39    1
46    1
12    1
55    1
96    1
7     1
90    1
6     1
55    1
78    1
36    1
57    1
15    1
16    1
77    1
78    1
72    1
43    1
Name: Intercept, dtype: float64

In [88]:
# Splitting intotraining and test data
ys2_train, ys2_test, xs2_train, xs2_test = train_test_split(ys2, xs2, random_state=0)

In [90]:
# Implementing trees
model_s2tr = tree.DecisionTreeClassifier()
model_s2tr.fit(xs2_train, ys2_train) 
predicts_s2tr = model_s2tr.predict(xs2_test)

In [91]:
print(metrics.classification_report(predicts_s2tr, ys2_test))
print(metrics.accuracy_score(ys2_test, predicts_s2tr ))     
print(metrics.confusion_matrix(ys2_test , predicts_s2tr))

             precision    recall  f1-score   support

        0.0       0.88      0.93      0.91       211
        1.0       0.42      0.27      0.33        37

avg / total       0.81      0.83      0.82       248

0.834677419355
[[197  27]
 [ 14  10]]


In [92]:
s2tr_fpr, s2_trtpr, thresholds = roc_curve(ys2_test, predicts_s2tr ,pos_label=1) 
s2tr_roc_auc = auc(s2tr_fpr, s2_trtpr)
s2tr_roc_auc


0.64806547619047616

In [ ]:
# Implementing KNN

In [93]:
model_s2nn = KNeighborsClassifier(n_neighbors = 50)
model_s2nn.fit(xs2_train, ys2_train) 
predicts_s2nn = model_s2nn.predict(xs2_test )
print(metrics.accuracy_score(ys2_test, predicts_s2nn))
print(metrics.confusion_matrix(ys2_test, predicts_s2nn))

0.903225806452
[[224   0]
 [ 24   0]]


In [94]:
s2nn_fpr, s2_nntpr, thresholds = roc_curve(ys2_test, predicts_s2nn ,pos_label=1) 
s2nn_roc_auc = auc(s2nn_fpr, s2_nntpr)
s2nn_roc_auc


0.5

In [ ]:
# Building and analyzing Data Scenario 3


In [25]:
# Adding label 0 to 2030 Dataset
year = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2030']
filepath = dhome+year[10]+".csv"
s3df=pd.read_csv(filepath)
grades = []
for row in s3df['ranks']:
    grades.append(0)
s3df['grades'] = grades



In [27]:
# Adding label 1 to the top 20 songs from 2006 to 2015


dhome = "/Users/avinesh/Google Drive/Courseworks/2 Sem/Python-based Data Analytics/Python Project/All Data/fsongs"
year = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2030']


for i in range(0,10):
    filepath = dhome+year[i]+".csv"
    tempdf1 = pandas.read_csv(filepath,header=0)
    grades = []
    for row in tempdf1['ranks']:
        if  row <= 20:
            grades.append(1)
        else:
            grades.append(0)
    tempdf1['grades'] = grades
    s3df = s3df.append(tempdf1)

In [28]:
print(s3df.shape)

19    0
19    1
19    1
19    1
19    1
19    1
19    1
19    1
19    1
19    1
19    1
Name: grades, dtype: int64
11
(1100, 23)


In [29]:
# Dropping Duplicates
s3od = s3df.sort(['songs','artists'], ascending=[1,0])
s3ot = s3od.drop_duplicates(['songs','artists'], keep='first')
print(s3ot.shape)
print(s3ot.columns.values)


(1009, 23)
['ranks' 'songs' 'artists' 'ids' 'acousticness' 'danceability' 'duration'
 'energy' 'key' 'liveness' 'loudness' 'mode' 'speechiness' 'tempo'
 'time_signature' 'valence' 'Anger' 'Surprise' 'Joy' 'Sadness' 'Love'
 'Fear' 'grades']


C:\Users\avinesh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [30]:
# Splitting into dependent and independent variables
ys3, xs3 = dmatrices('grades ~ acousticness + danceability + duration + energy + key + liveness + loudness + mode + speechiness + tempo + time_signature + valence + Anger + Surprise + Love + Joy + Sadness + Fear', 
                   data = s3ot, return_type="dataframe")
ys3 = numpy.ravel(ys3)

In [31]:
xs3.pop('Intercept')

13    1
96    1
60    1
64    1
77    1
77    1
70    1
89    1
86    1
22    1
85    1
17    1
80    1
40    1
20    1
91    1
55    1
42    1
50    1
86    1
54    1
96    1
99    1
96    1
46    1
31    1
5     1
32    1
7     1
68    1
     ..
95    1
48    1
99    1
6     1
31    1
80    1
38    1
81    1
16    1
79    1
40    1
13    1
73    1
69    1
39    1
46    1
12    1
55    1
96    1
7     1
90    1
6     1
55    1
78    1
36    1
57    1
15    1
77    1
78    1
43    1
Name: Intercept, dtype: float64

In [32]:
xs3.columns.values

array(['acousticness', 'danceability', 'duration', 'energy', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'Anger', 'Surprise', 'Love', 'Joy',
       'Sadness', 'Fear'], dtype=object)

In [33]:
# Splitting into test and training data
ys3_train, ys3_test, xs3_train, xs3_test = train_test_split(ys3, xs3, random_state=0)

In [34]:
# Implementing logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
model_lrs3 = LogisticRegression()
model_lrs3.fit(xs3_train, ys3_train)  

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
model_lrs3.coef_

array([[ -3.97091269e-02,   1.33863240e+00,  -1.15945165e-03,
         -8.67564450e-01,   3.47791559e-02,   6.67221676e-01,
         -1.72543146e-02,  -3.13527475e-01,  -9.58252862e-01,
         -3.07872240e-04,  -4.99918524e-02,   5.43439870e-01,
         -1.15588011e-01,   1.72058063e-01,  -1.40505379e-01,
         -1.74636377e-01,  -8.24457035e-02,  -1.73964713e-01]])

In [36]:
xs3_train.columns.values

array(['acousticness', 'danceability', 'duration', 'energy', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'Anger', 'Surprise', 'Love', 'Joy',
       'Sadness', 'Fear'], dtype=object)

In [114]:
# Implementing trees
model_s3tr = tree.DecisionTreeClassifier()
model_s3tr.fit(xs3_train, ys3_train) 
predicts_s3tr = model_s3tr.predict(xs3_test)

In [115]:
print(metrics.classification_report(predicts_s3tr, ys3_test))
print(metrics.accuracy_score(ys3_test, predicts_s3tr ))     
print(metrics.confusion_matrix(ys3_test , predicts_s3tr))

             precision    recall  f1-score   support

        0.0       0.79      0.81      0.80       199
        1.0       0.24      0.22      0.23        54

avg / total       0.68      0.68      0.68       253

0.683794466403
[[161  42]
 [ 38  12]]


In [116]:
s3tr_fpr, s3_trtpr, thresholds = roc_curve(ys3_test, predicts_s3tr ,pos_label=1) 
s3tr_roc_auc = auc(s3tr_fpr, s3_trtpr)
s3tr_roc_auc


0.51655172413793105

In [123]:
# Implementing K Nearest Neighbors
model_s3nn = KNeighborsClassifier(n_neighbors = 10)
model_s3nn.fit(xs3_train, ys3_train) 
predicts_s3nn = model_s3nn.predict(xs3_test )
print(metrics.accuracy_score(ys3_test, predicts_s3nn))
print(metrics.confusion_matrix(ys3_test, predicts_s3nn))

0.802371541502
[[203   0]
 [ 50   0]]


In [125]:
s3nn_fpr, s3_nntpr, thresholds = roc_curve(ys3_test, predicts_s3nn ,pos_label=1) 
s3nn_roc_auc = auc(s3nn_fpr, s3_nntpr)
s3nn_roc_auc


0.5

In [ ]:
# Building and analyzing Data Scenario 4

In [126]:
dhome = "/Users/avinesh/Google Drive/Courseworks/2 Sem/Python-based Data Analytics/Python Project/All Data/fsongs"
year = ['2030','2020','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
# Adding label 1 to 2020 Dataset
filepath = dhome+year[1]+".csv"
df=pd.read_csv(filepath)
grades = []


In [133]:
df

,ranks,songs,artists,ids,acousticness,danceability,duration,energy,key,liveness,...,tempo,time_signature,valence,Anger,Surprise,Joy,Sadness,Love,Fear,grades
0,1,THE TWIST,Chubby Checker,SOKPTVF13725C7344B,0.035865,0.466492,159.55592,0.647644,4,0.056933,...,155.230,4,0.966031,0.727710,0.783294,0.783294,0.715073,0.903446,0.783294,1
1,2,SMOOTH,santana,SOLHYBC135CC0B27FE,0.170286,0.620632,298.06667,0.922940,9,0.093313,...,115.952,4,0.962458,1.357096,1.428605,1.437286,1.335036,1.570031,1.437286,1
2,3,MACK THE KNIFE,Bobby Darin,SOYAUIR1376B0571F2,0.701482,0.546443,178.62531,0.852067,8,0.905568,...,95.200,4,0.634315,1.605895,1.684853,1.684853,1.632401,1.826029,1.684853,1
3,4,HOW DO I LIVE,LeAnn Rimes,SODLOLZ144912CCD9E,0.005334,0.764592,237.26621,0.908486,11,0.058760,...,127.972,4,0.886589,0.910407,0.981885,0.984372,0.879784,1.101369,0.984372,1
4,5,PARTY ROCK ANTHEM,lmfao,SOOIYXQ14373B73954,0.060660,0.683526,376.63156,0.501430,5,0.105768,...,130.033,4,0.182814,1.259442,1.340614,1.340614,1.182032,1.519476,1.340614,1
5,6,I GOTTA FEELING,The Black Eyed Peas,SOBTUCX146012A297D,0.083967,0.743047,289.13197,0.740592,0,0.421652,...,127.954,4,0.652845,1.439891,1.537006,1.537006,1.410294,1.695163,1.537006,1
6,7,macarena,Los Del Rio,SOVWGXW13D531FB427,0.005155,0.609297,220.31628,0.926622,0,0.163401,...,134.995,4,0.773814,1.182268,1.159297,1.261077,1.173461,1.283312,1.210187,1
7,8,PHYSICAL,Olivia Newton-John,SODITYF135C40313F7,0.100817,0.827085,222.62667,0.830078,4,0.109286,...,124.285,4,0.806108,1.004527,1.095194,1.095194,0.931961,1.239927,1.095194,1
8,9,YOU LIGHT UP MY LIFE,Debby Boone,SORVCNN13737B9E003,0.855203,0.322341,218.93333,0.282051,2,0.162640,...,77.470,3,0.078563,1.238923,1.281668,1.287147,1.261322,1.376655,1.287147,1
9,10,HEY JUDE,The Beatles,SOKMJYC13678934206,0.028010,0.402092,431.33333,0.551766,10,0.092866,...,145.164,4,0.651103,0.805659,0.850873,0.850873,0.798673,0.922265,0.850873,1


In [128]:
for row in df['ranks']:
    grades.append(1)
    
df['grades'] = grades


In [130]:

#Adding label 0 to 2030 Dataset
filepath = dhome+year[0]+".csv"
df1=pd.read_csv(filepath)
grades = []
for row in df1['ranks']:
    grades.append(0)

df1['grades'] = grades

df = df.append(df1)



In [132]:

# Adding label 0 to Remaining Dataset
for i in range(2,12):
    filepath = dhome+year[i]+".csv"
    mds = pd.read_csv(filepath,header=0)
    grades = []
    for row in mds['ranks']:
        if  row > 90:
            grades.append(1)
        else:
            grades.append(0)
    mds['grades'] = grades
    df = df.append(mds[91:])


In [134]:


print(list(df.columns.values))


['ranks', 'songs', 'artists', 'ids', 'acousticness', 'danceability', 'duration', 'energy', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'Anger', 'Surprise', 'Joy', 'Sadness', 'Love', 'Fear', 'grades']


In [149]:
# Dropping Duplicates
s4od = df.sort(['songs','artists'], ascending=[1,0])
s4ot = s4od.drop_duplicates(['songs','artists'], keep='first')
print(s4ot.shape)
print(s4ot.columns.values)


(292, 23)
['ranks' 'songs' 'artists' 'ids' 'acousticness' 'danceability' 'duration'
 'energy' 'key' 'liveness' 'loudness' 'mode' 'speechiness' 'tempo'
 'time_signature' 'valence' 'Anger' 'Surprise' 'Joy' 'Sadness' 'Love'
 'Fear' 'grades']


C:\Users\avinesh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [150]:
# Splitting into dependent and independent variables
ys4, xs4 = dmatrices('grades ~ -1 + acousticness + danceability + duration + energy + key + liveness + loudness + mode + speechiness + tempo + time_signature + valence + Anger + Surprise + Love + Joy + Sadness + Fear', 
                   data = s4ot, return_type="dataframe")
ys4 = numpy.ravel(ys4)

In [151]:
xs4

,acousticness,danceability,duration,energy,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,Anger,Surprise,Love,Joy,Sadness,Fear
19,0.074356,0.517933,393.60000,0.492540,1,0.069184,-7.396,1,0.027861,131.189,4,0.235759,1.261152,1.308458,1.436487,1.319803,1.268168,1.319803
13,0.074356,0.517933,393.60000,0.492540,1,0.069184,-7.396,1,0.027861,131.189,4,0.235759,1.275426,1.323098,1.451504,1.334373,1.283369,1.334373
65,0.473817,0.511773,242.36000,0.823738,9,0.080829,-7.269,1,0.273506,148.063,3,0.183434,1.645729,1.748974,2.039133,1.784766,1.519658,1.784766
96,0.656259,0.655754,274.12000,0.584733,11,0.576725,-8.176,0,0.355979,88.487,4,0.444674,1.712066,1.835757,1.965904,1.804292,1.679489,1.806791
17,0.004035,0.682043,201.77333,0.987871,2,0.283666,-3.859,1,0.035419,140.037,4,0.794043,1.512821,1.669518,1.769517,1.628808,1.446962,1.613404
91,0.024306,0.600530,216.49333,0.880661,1,0.064578,-4.892,0,0.031792,107.020,4,0.578424,1.247060,1.286293,1.440836,1.313338,1.214792,1.313338
87,0.007207,0.784116,306.29288,0.664036,9,0.158843,-8.482,0,0.034874,127.549,4,0.961100,1.556438,1.616655,1.779251,1.620672,1.545891,1.620672
96,0.000327,0.501678,225.73333,0.846119,0,0.083094,-4.885,1,0.033566,137.954,4,0.532250,0.936032,0.992708,1.121548,1.002639,0.897349,1.002639
99,0.053297,0.549344,245.91125,0.577499,0,0.210349,-7.984,1,0.033878,119.701,4,0.172388,0.850268,0.853724,0.967070,0.879516,0.813215,0.879516
96,0.137173,0.559591,196.10667,0.603488,2,0.113865,-5.208,1,0.270665,179.083,4,0.207714,1.129402,1.171097,1.358733,1.201678,1.070920,1.201678


In [152]:
# Splitting into test and training data 
ys4_train, ys4_test, xs4_train, xs4_test = train_test_split(ys4, xs4, random_state=0)

In [153]:
# Implementing trees
model_s4tr = tree.DecisionTreeClassifier()
model_s4tr.fit(xs4_train, ys4_train) 
predicts_s4tr = model_s4tr.predict(xs4_test)

In [154]:
print(metrics.classification_report(predicts_s4tr, ys4_test))
print(metrics.accuracy_score(ys4_test, predicts_s4tr ))     
print(metrics.confusion_matrix(ys4_test , predicts_s4tr))

             precision    recall  f1-score   support

        0.0       0.58      0.44      0.50        34
        1.0       0.60      0.72      0.65        39

avg / total       0.59      0.59      0.58        73

0.58904109589
[[15 11]
 [19 28]]


In [155]:
s4tr_fpr, s4_trtpr, thresholds = roc_curve(ys4_test, predicts_s4tr ,pos_label=1) 
s4tr_roc_auc = auc(s4tr_fpr, s4_trtpr)
s4tr_roc_auc


0.58633387888707034

In [156]:
# Implementing NN
model_s4nn = KNeighborsClassifier(n_neighbors = 50)
model_s4nn.fit(xs4_train, ys4_train) 
predicts_s4nn = model_s4nn.predict(xs4_test )
print(metrics.accuracy_score(ys4_test, predicts_s4nn))
print(metrics.confusion_matrix(ys4_test, predicts_s4nn))

0.643835616438
[[ 0 26]
 [ 0 47]]


In [157]:
s4nn_fpr, s4nn_tpr, thresholds = roc_curve(ys4_test, predicts_s4nn ,pos_label=1) 
s4nn_roc_auc = auc(s4nn_fpr, s4nn_tpr)
s4nn_roc_auc


0.5

In [ ]:
# Building and analyzing Data Scenario 5


In [159]:
dhome = "/Users/avinesh/Google Drive/Courseworks/2 Sem/Python-based Data Analytics/Python Project/All Data/fsongs"
year = ['2030','2020','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
# Creating an empty dataframe 
df = pd.DataFrame()
grades = []


In [161]:

# Adding label 0 to Remaining Dataset
for i in range(2,12):
    filepath = dhome+year[i]+".csv"
    mds = pd.read_csv(filepath, header=0)
    grades = []
    for row in mds['ranks']:
        if  row < 20:
            grades.append(1)
        else:
            grades.append(0)
    mds['grades'] = grades
    df = df.append(mds)


In [162]:
df

,ranks,songs,artists,ids,acousticness,danceability,duration,energy,key,liveness,...,tempo,time_signature,valence,Anger,Surprise,Joy,Sadness,Love,Fear,grades
0,1,BAD DAY,Daniel Powter,SOCUEKL12A6701F24D,0.448091,0.598891,233.64000,0.785222,3,0.151361,...,140.046,4,0.533984,1.144925,1.218268,1.224747,1.115529,1.340507,1.221507,1
1,2,TEMPERATURE,Sean Paul,SONOROL13772EE7BC6,0.094097,0.853728,215.18667,0.723126,10,0.294713,...,125.055,4,0.800229,1.278748,1.346167,1.341026,1.281008,1.449488,1.341026,1
2,3,PROMISCUOUS,nelly furtado,SOHOVEJ14D5DA5E5A5,0.280851,0.759863,136.20000,0.751573,10,0.037555,...,113.221,4,0.791129,1.329249,1.414882,1.414882,1.297656,1.564637,1.414882,1
3,4,YOU'RE BEAUTIFUL,James Blunt,SOATYZP1371B78DF20,0.631469,0.673684,212.96000,0.544647,0,0.088874,...,81.981,4,0.436098,1.296739,1.388265,1.379763,1.276330,1.509597,1.379763,1
4,5,HIPS DON'T LIE,shakira,SOOLHVK13739436E66,0.771004,0.388608,233.79546,0.416492,6,0.629153,...,84.698,7,0.835740,1.593427,1.726447,1.718780,1.547437,1.918055,1.718780,1
5,6,UNWRITTEN,Natasha Bedingfield,SOLJPJK144913EF650,0.989982,0.396345,202.54766,0.271146,5,0.966229,...,64.592,3,0.736081,1.288989,1.370622,1.370622,1.277504,1.505588,1.370622,1
6,7,CRAZY,Gnarls Barkley,SOBKMKP14509A7F36E,0.050496,0.873095,180.53333,0.579954,0,0.089438,...,111.948,4,0.645920,1.447293,1.550609,1.559917,1.375200,1.732372,1.559917,1
7,8,RIDIN',chamillionaire,SOXCRBA144DD7FC4F4,0.236065,0.735952,290.69333,0.894893,6,0.316937,...,127.575,4,0.660509,1.008412,1.193232,1.069758,0.970334,1.195916,1.069007,1
8,9,SEXYBACK,Justin Timberlake,SOQJIUO13E8AD605E0,0.064191,0.954246,232.90667,0.650252,10,0.038787,...,120.011,4,0.940339,1.244730,1.250011,1.302647,1.190905,1.455581,1.302647,1
9,10,CHECK ON IT,beyonce,SOIUZUJ13CF69DBC5C,0.000288,0.549517,27.22667,0.798370,2,0.316043,...,83.201,4,0.701100,1.054108,1.124883,1.121963,1.036244,1.229770,1.121963,1


In [163]:
# Dropping Duplicates
s5od = df.sort(['songs','artists'], ascending=[1,0])
s5ot = s5od.drop_duplicates(['songs','artists'], keep='first')
print(s5ot.shape)
print(s5ot.columns.values)

(909, 23)
['ranks' 'songs' 'artists' 'ids' 'acousticness' 'danceability' 'duration'
 'energy' 'key' 'liveness' 'loudness' 'mode' 'speechiness' 'tempo'
 'time_signature' 'valence' 'Anger' 'Surprise' 'Joy' 'Sadness' 'Love'
 'Fear' 'grades']


C:\Users\avinesh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [164]:
# Splitting into dependent and independent variables
ys5, xs5 = dmatrices('grades ~ -1 + acousticness + danceability + duration + energy + key + liveness + loudness + mode + speechiness + tempo + time_signature + valence + Anger + Surprise + Love + Joy + Sadness + Fear', 
                   data = s4ot, return_type="dataframe")
ys5 = numpy.ravel(ys5)

In [166]:
# Splitting into test and training data
ys5_train, ys5_test, xs5_train, xs5_test = train_test_split(ys5, xs5, random_state=0)

In [167]:
# Implementing trees
model_s5tr = tree.DecisionTreeClassifier()
model_s5tr.fit(xs5_train, ys5_train) 
predicts_s5tr = model_s5tr.predict(xs5_test)

In [168]:
print(metrics.classification_report(predicts_s5tr, ys5_test))
print(metrics.accuracy_score(ys5_test, predicts_s5tr ))     
print(metrics.confusion_matrix(ys5_test , predicts_s5tr))

             precision    recall  f1-score   support

        0.0       0.54      0.42      0.47        33
        1.0       0.60      0.70      0.64        40

avg / total       0.57      0.58      0.57        73

0.575342465753
[[14 12]
 [19 28]]


In [169]:
s5tr_fpr, s5_trtpr, thresholds = roc_curve(ys5_test, predicts_s5tr ,pos_label=1) 
s5tr_roc_auc = auc(s5tr_fpr, s5_trtpr)
s5tr_roc_auc


0.56710310965630106

In [170]:
# Implementing K Nearest Neighbors
model_s5nn = KNeighborsClassifier(n_neighbors = 50)
model_s5nn.fit(xs5_train, ys5_train) 
predicts_s5nn = model_s5nn.predict(xs5_test )
print(metrics.accuracy_score(ys5_test, predicts_s5nn))
print(metrics.confusion_matrix(ys5_test, predicts_s5nn))

0.643835616438
[[ 0 26]
 [ 0 47]]


In [171]:
s5nn_fpr, s5nn_tpr, thresholds = roc_curve(ys5_test, predicts_s5nn ,pos_label=1) 
s5nn_roc_auc = auc(s5nn_fpr, s5nn_tpr)
s5nn_roc_auc


0.5